# CPS "Pop Stats" Socioeconomic Files

Here we simply take the PDFs provided by Open City and parse them into CSV files. 

In [10]:
import PyPDF2
import os
import pandas as pd
import numpy as np
from tabula import read_pdf

We first split the first two years PDFs into two seperate documents, as the content switches from raw values to percentages between even and odd pages. 

In [3]:
def splitPDF(file_path):
    ''' given file_path, opens pdf, splits into even and odd pages, writes both to respective folders '''
    # open pdf reader object (file path exists by construction)
    pdf = PyPDF2.PdfFileReader(open(file_path, "rb"))
    
    
    # writer objects for storing odd and even pages
    odd_pages = PyPDF2.PdfFileWriter()
    even_pages = PyPDF2.PdfFileWriter()
    
    for page_num in range(0, pdf.numPages):
        # if page number is odd
        if (page_num + 1) % 2 != 0:
            odd_pages.addPage(pdf.getPage(page_num))
        else:
            even_pages.addPage(pdf.getPage(page_num))
            
            
    # get just file name, splitting / to remove directory
    
    (directory, file_name) = file_path.split("/")
    # odd pages are actual number estimates
    with open("{}/Numbers - {}".format(directory, file_name), "wb") as outputStream: #create new PDF
        odd_pages.write(outputStream)
        
    # even pages are percentage estimates
    with open("{}/Percentages - {}".format(directory, file_name), "wb") as outputStream:
        even_pages.write(outputStream)

In [5]:
# get all census files from 2010 to 2011 stored in Census Tract Summaries folder (ignoring first .dstore)
file_names = ["Census Tract Summaries/" + file for file in os.listdir("Census Tract Summaries")[1:]]

# split first two pdfs
for file in file_names[:2]:
    splitPDF(file)

In [6]:
# get new file_names
file_names = ["Census Tract Summaries/" + file for file in os.listdir("Census Tract Summaries")[1:]]

We now use the tabula package to quickly read the pdf tables into a dataframe (note: the package is a simple wrapper function for a java library, so java is a requirement to use it). 

Unfortunately, given the formatting, this process works best if you give the feed the package an "area" tuple, detailing what section of the document actually contains the data. Specifically, we open each PDF in MacOS Preview, and use the crop tool to select the table's area. We note the areas below in the following list.

Similarly, we ignore the provided header, which gets incorrectly read as multiple rows, and manually provide them for each census. 

In [26]:
# helper function for converting Preview's coords to area tuple
get_area = lambda left, top, width, height: (top, left, top + height, left+width)

area2010_numbers = get_area(141, 45.19, 346.47, 691.47)
area2011_numbers = get_area(136.88, 56.6, 296.15, 681.24)
area_percent_2011 = get_area(134.19, 103.01, 295.91, 586.04)
area2012 = get_area(15.37, 69.96, 730.77, 525.55)
area2013 = get_area(35.89, 159.18, 1602.76, 1189.47)
area2014 = get_area(37.66, 159.73, 1565.8, 1182.17)
area2015 = get_area(34.84, 159.39, 1569.27, 1186.13)
area2016 = get_area(16.27, 70.9, 736.39, 528.64)
area2017 = get_area(15.13, 71.5, 738.09, 527.02)

In [8]:
column_names_2010 = ["Tract", "Estimated Median Family Income", "Less Than High School", "High School", "Some College",
                "College Graduate", "Graduate School", "Educational Attainment Score", "% of Single Parent-Households"
                , "% of Owner Occupied Homes", "% of Population Speaking a Language Other than English", "Number of School-Age Children"]

census2010_numbers = read_pdf(file_names[0], pages= "all", area = area2010_numbers, 
                      pandas_options = {'header': None, 'names': column_names_2010, 'index_col': "Tract"})

In [12]:
column_names_2011 = ["Tract", "Estimated Median Family Income", "Less Than High School", "High School", "Some College",
                "College Graduate", "Graduate School", "Educational Attainment Score", "% of Single Parent-Households"
                , "% of Owner Occupied Homes", "% of Population Speaking a Language Other than English", 
                     "Weighted Average ISAT Performance at Attendance Area Schools", "Number of School-Age Children"]

census2011_numbers = read_pdf(file_names[1], pages = "all", area = area2011_numbers, 
                      pandas_options = {'header': None, 'names': column_names_2011, 'index_col': 'Tract'})

In [25]:
percent_columns = ["Tract", "Estimated Median Family Income Percentile", "Educational Attainment Score Percentile",
                   "% of Single Parent Households Percentile", "% of Owner Occupied Homes Percentile", 
                   "% of Population Speaking a Language Other than English Percentile", 
                   "5-Factor Socioeconomic Score", "5-Factor Socioeconomic Tier"]

census2010_percent = read_pdf(file_names[2], pages = "all", pandas_options = 
                              {'header': None, 'names': percent_columns, 'index_col': 'Tract'})

KeyError: 'Tract'

In [27]:
percent_2011 = ["Tract", "Estimated Median Family Income Percentile", "Educational Attainment Score Percentile",
                   "% of Single Parent Households Percentile", "% of Owner Occupied Homes Percentile", 
                   "% of Population Speaking a Language Other than English Percentile", 
                   "Weighted Average ISAT Performance at Attendance Area Schools Percentile", 
                   "6-Factor Socioeconomic Score", "6-Factor Socioeconomic Tier"]



census2011_percent = read_pdf(file_names[3], pages = "all", area = area_percent_2011, pandas_options = 
                              {'header': None, "names": percent_2011, 'index_col': "Tract"})

KeyError: 'Tract'

The files from now on are combined and contain every column from both the numbers and percent files from the previous year. 

In [20]:
# copy columns, extend with percentages
column_names_2012 = column_names_2011[:]
column_names_2012.extend(percent_2011[1:])

column_names_2012 = column_names_2012[:21]

census2012 = read_pdf(file_names[4], pages = "all", area = area2012, pandas_options = 
                      {'header': None, 'names': column_names_2012, 'index_col': 'Tract'})

In [21]:
census2013 = read_pdf(file_names[5], pages = 'all', area = area2013, pandas_options = 
                      {'header': None, 'names': column_names_2012, 'index_col': 'Tract'})

census2014 = read_pdf(file_names[6], pages = 'all', area = area2014, pandas_options = 
                      {'header': None, 'names': column_names_2012, 'index_col': 'Tract'})

From 2015 on, the performance metric switches from ISAT to NWEA, so the columns must also change. 

In [22]:
column_names_2015 = column_names_2012[:]

column_names_2015[11] = column_names_2015[11].replace("ISAT", "NWEA")
column_names_2015[-3] = column_names_2015[-3].replace("ISAT", "NWEA")

In [24]:
census2015 = read_pdf(file_names[7], pages = 'all', area = area2015, pandas_options = 
                      {'header': None, 'names': column_names_2015, 'index_col': 'Tract'})

census2016 = read_pdf(file_names[8], pages = 'all', area = area2016, pandas_options = 
                      {'header': None, 'names': column_names_2015, 'index_col': 'Tract'})

census2017 = read_pdf(file_names[9], pages = 'all', area = area2017, pandas_options = 
                      {'header': None, 'names': column_names_2015, 'index_col': 'Tract'})

In [ ]:
census2010 = pd.merge(census2010_numbers, census2010_percent, left_index = True, right_index = True)

#### Save files for external use. 

In [ ]:

census2010.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2010-2011.csv")
census2011.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2011-2012.csv")
census2012.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2012-2013.csv")
census2013.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2013-2014.csv")
census2014.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2014-2015.csv")
census2015.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2015-2016.csv")
census2016.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2016-2017.csv")
census2017.to_csv("Census Tract CSVs/Summary of Census Socioeconomic Data 2017-2018.csv")